In [53]:
#import libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

In [54]:
#loading dataset
diabetes = pd.read_csv('diabetes.csv')  
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [55]:
diabetes.head() #quick look of the dataset

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [56]:
768*0.8

614.4000000000001

In [57]:
#split training and testing set 0.8/0.2
train_set = diabetes.iloc[0:614,:]
test_set = diabetes.iloc[614:,:]

In [58]:
#split the features and labels
X_train = train_set.iloc[:,:-1].to_numpy()
y_train = train_set.iloc[:,-1].to_numpy()
X_test = test_set.iloc[:,:-1].to_numpy()
y_test = test_set.iloc[:,-1].to_numpy()
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(614, 8)
(614,)
(154, 8)
(154,)


In [59]:
#perceptron 
#ref.https://www.youtube.com/watch?v=t2ym2a3pb_Y
class Perceptron:
    def __init__(self, learning_rate = 0.01, n_iters = 1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        n_samples, n_features = X.shape

        # weights initialization
        self.weights = np.zeros(n_features)
        self.bias = 0

        y_ = np.array([1 if i > 0 else 0 for i in y])

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)

                update = self.lr * (y_[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update 

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted

    def _unit_step_func(self, x):
        return np.where(x>=0, 1, 0)

In [60]:
p = Perceptron(learning_rate = 0.01, n_iters = 1000)#default parameter setting

#model training
p.fit(X_train, y_train)

y_pred = p.predict(X_test)

#model evaluation
accuracy = np.sum(y_test == y_pred) / len(y_test)
print('The accuracy of the Perceptron is', accuracy*100, '%')

The accuracy of the Perceptron is 50.649350649350644 %


In [61]:
#MLP model testing

#random seed setting
np.random.seed(7)

#Splitting the input variable x and the output variable y
features= diabetes.iloc[:,:-1].to_numpy()#all features
labels= diabetes.iloc[:,-1].to_numpy() #all labels

#modelling
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu')) #12 hidden neurons, 8 dimensions for the 8 features
model.add(Dense(8, activation='relu')) #8 neurons for hidden layer
model.add(Dense(1, activation='sigmoid')) #1 neuron for output layer

#model compiling
#The compiled model is designed to make the model more efficient using the numerical calculations encapsulated by Keras
#parameter setting:
#The loss function to evaluate a set of weights must be specified when the model is compiled
#Optimizer for searching the network with different weights (optimizer)（optimizer）
#Binary classification is defined as binary cross-entropy
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

#model training
model.fit(x=features,y=labels,epochs=150,batch_size=10) #number of iterations 150

#model evaluation
scores= model.evaluate(x=features,y=labels)
print('\n The accuracy of the Multiple Layers Perceptron', scores[1]*100, '%')

Epoch 1/150
77/77 [==============================] - 0s 1ms/step - loss: 4.9115 - accuracy: 0.4180
Epoch 2/150
77/77 [==============================] - 0s 1ms/step - loss: 2.0495 - accuracy: 0.4727
Epoch 3/150
77/77 [==============================] - 0s 1ms/step - loss: 1.1055 - accuracy: 0.5521
Epoch 4/150
77/77 [==============================] - 0s 1ms/step - loss: 0.8846 - accuracy: 0.5846
Epoch 5/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7778 - accuracy: 0.6094
Epoch 6/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7317 - accuracy: 0.6367
Epoch 7/150
77/77 [==============================] - 0s 1ms/step - loss: 0.7061 - accuracy: 0.6419
Epoch 8/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6941 - accuracy: 0.6445
Epoch 9/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6749 - accuracy: 0.6602
Epoch 10/150
77/77 [==============================] - 0s 1ms/step - loss: 0.6597 - accuracy: 0.6628
Epoch 11/

In [62]:
#increasing the number of iterations 
#model training
model.fit(x=features,y=labels,epochs=500,batch_size=10)

#model evaluation
scores= model.evaluate(x=features,y=labels)
print('\n The accuracy of the Multiple Layers Perceptron', scores[1]*100, '%')

Epoch 1/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4954 - accuracy: 0.7656
Epoch 2/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4830 - accuracy: 0.7695
Epoch 3/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4800 - accuracy: 0.7747
Epoch 4/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4848 - accuracy: 0.7773
Epoch 5/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4802 - accuracy: 0.7734
Epoch 6/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4776 - accuracy: 0.7734
Epoch 7/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4744 - accuracy: 0.7721
Epoch 8/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4744 - accuracy: 0.7773
Epoch 9/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4781 - accuracy: 0.7799
Epoch 10/500
77/77 [==============================] - 0s 1ms/step - loss: 0.4814 - accuracy: 0.7682
Epoch 11/

In [63]:
#continue increasing the number of iterations 
#model training
model.fit(x=features,y=labels,epochs=2000,batch_size=10)

#model evaluation
scores= model.evaluate(x=features,y=labels)
print('\n The accuracy of the Multiple Layers Perceptron', scores[1]*100, '%')

Epoch 1/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4203 - accuracy: 0.7956
Epoch 2/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4245 - accuracy: 0.7995
Epoch 3/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4212 - accuracy: 0.7969
Epoch 4/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4228 - accuracy: 0.7969
Epoch 5/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4238 - accuracy: 0.7995
Epoch 6/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4263 - accuracy: 0.7878
Epoch 7/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4252 - accuracy: 0.7930
Epoch 8/2000
77/77 [==============================] - 0s 1ms/step - loss: 0.4218 - accuracy: 0.8086
Epoch 9/2000
77/77 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.8112
Epoch 10/2000
77/77 [==============================] - 0s 2ms/step - loss: 0.4219 - accuracy: 0.8021